# M.Lytova, M.Spanner, I.Tamblyn. *Deep learning and high harmonic generation* (2020)
## Codes for Section IV.C : *Latent space visualization for VAE*

### see also (for beta=1) https://github.com/emnajaoua/beta_variational_autoencoders/blob/master/disentangled_vae%20(1).ipynb 

##Headers and constants

In [ ]:
from google.colab import files
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Dropout, Lambda
from keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Nadam, Adam
from tensorflow.keras import initializers
from keras import objectives
from keras.losses import mean_squared_error
from keras.callbacks import TensorBoard
from keras import backend as K
import argparse
import matplotlib.pyplot as plt
import time

In [ ]:
PI = 3.14159265359

t_n_points = 4096        # number of nodes in time
t_n = np.linspace(0, 800, t_n_points)/41.341   # grid in time, Tmax = 800 a.u. = 19.35 fs

# VAE parameters
batch_size = 32       
latent_dim = 2
epsilon_std = 0.01

n_train = batch_size * 600   # training set size
n_test = batch_size * 25     # testing set size

##Loading a training set

In [ ]:
param_train = np.zeros((n_train, 3))
path2param = f"/hhg_ThI/param.dat"
param_train = np.loadtxt(path2param, delimiter = ",", max_rows = n_train) 

In [ ]:
y_train = np.zeros((n_train, t_n_points))
path2load0 = f"/hhg_ThI/hhg"

tic = time.perf_counter()

for i in range(n_train): 
    path2load = path2load0 + str(i+1) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_train[i] = load_data[0:t_n_points] * np.sin(PI*t_n/Tmax) 
    if (round(i/1000)==i/1000):
        print(i)        

toc = time.perf_counter()
print(f"Training set preparation time {toc - tic:0.4f} seconds")   

In [ ]:
def plot_train_example(i):
    plt.figure(figsize=(12,4), constrained_layout=False)    
    plt.plot(t_n, y_train[i], color='green')
    plt.title('theta = ' + str(round(param_train[i,0]*180/PI,2)) + ", I = " + str(round((param_train[i,1]/5.338027e-2)**2,2)) + "e14 W/cm^2, R = 2 a.u.", fontsize=16)
    plt.xlabel('$t$', fontsize=14)
    plt.ylabel('$y(t)$', fontsize=14)
    plt.xticks(np.arange(0, Tmax, 2.0))
    plt.grid()    
    plt.show() 
    plt.close()

### Drawing of a randomly chosen $d_k(t)$

In [ ]:
i_show = np.random.randint(0, n_train-1)
plot_train_example(i_show)

##Loading a testing set

In [ ]:
param_test = np.zeros((n_test, 3))
path2param = f"/hhg_ThI/param.dat"
param_test = np.loadtxt(path2param, delimiter = ",", skiprows = n_train, max_rows = n_test) 

In [ ]:
y_test = np.zeros((n_test, t_n_points))
path2load0 = f"/hhg_ThI/hhg"

for i in range(n_test): 
    path2load = path2load0 + str(i+1+n_train) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_test[i] = load_data[0:t_n_points] * np.sin(PI*t_n/Tmax)     

##Normalizing before trainig

In [ ]:
max_E0 = np.amax(param_train[:,1])
min_E0 = np.amin(param_train[:,1])
param_train_norm = (param_train-[0, min_E0])/[PI/2, (max_E0-min_E0)]
param_test_norm = (param_test-[0, min_E0])/[PI/2, (max_E0-min_E0)]
y_train_max = np.amax(abs(y_train))
y_test_max = np.amax(abs(y_test))
y_max = max(y_train_max, y_test_max)
y_train_norm = y_train/y_max
y_test_norm = y_test/y_max

##Latent space vectors, encoder and decoder

In [ ]:
# Generate the latent representation vectors 
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_sigma) * epsilon

In [ ]:
# Encoder

inputs = Input(shape = t_n_points, name='encoder_input')

x = Dense(512, activation='tanh')(inputs)
x = Reshape((512, 1))(x) 
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x) 
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x) 
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x) 
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x) 
x = Conv1D(1, 4, activation='tanh', padding='same')(x)
x = Flatten()(x)
x = Dense(64, activation='tanh')(x)
x = Dense(16, activation='tanh')(x)

z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

print(encoder.summary())

In [ ]:
# Decoder

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')

x = Dense(16, activation='tanh')(latent_inputs) 
x = Dense(64, activation='tanh')(x) 
x = Reshape((64, 1))(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(8, 4, activation='tanh', padding='same')(x)  
x = Conv1D(8, 4, activation='tanh', padding='same')(x) 
x = UpSampling1D(2)(x)  
x = Conv1D(8, 4, activation='tanh', padding='same')(x)                                                                      
x = Conv1D(1, 4, activation='tanh', padding='same')(x)
x = Flatten()(x)
x = Dense(512, activation='tanh')(x) 

outputs = Dense(t_n_points, activation='tanh')(x)

decoder = Model(latent_inputs, outputs, name='decoder')

print(decoder.summary())

In [ ]:
# build the model
outputs = decoder(encoder(inputs)[2]) 

vae = Model(inputs, outputs, name='vae_mlp')

In [ ]:
# loss function for VAE and compiling the model

reconstruction_loss = mean_squared_error(inputs, outputs)

reconstruction_loss *= t_n_points

kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)

kl_loss = K.sum(kl_loss, axis=-1)

kl_loss *= -0.5

vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)

opt = Adam(lr=0.0005, amsgrad=True)
vae.compile(optimizer=opt)

print(vae.summary())

##Training

In [ ]:
def plot_losses2():
    plt.figure(figsize=(8,4))
    plt.plot(np.log10(loss),color='blue')
    plt.plot(np.log10(val_loss),color='red')
    plt.ylabel('log(Loss)', fontsize=12)
    plt.xlabel('Epoch', fontsize=12)
    plt.legend(['training', 'validation'], loc='upper right', fontsize=14)
    plt.show()

In [ ]:
tic = time.perf_counter()

history = vae.fit(y_train_norm, 
                epochs=200,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(y_test_norm, None))

toc = time.perf_counter()
print(f"Execution time {toc - tic:0.4f} seconds")

path = f"/auto_thetaI/model_2" 
vae.save(path) 

path1 = f"/auto_thetaI/encoder_2" 
encoder.save(path1) 

path2 = f"/auto_thetaI/decoder_2" 
decoder.save(path2)

loss = history.history['loss']
val_loss = history.history['val_loss']

plot_losses2()

##Training and validation losses

In [ ]:
plot_losses2() 

##Prediction

In [ ]:
z_mean, _, _ = encoder.predict(y_test_norm, batch_size=batch_size)
prediction_norm = decoder.predict(z_mean)
prediction = prediction_norm * y_max

## Drawing the latent space

In [ ]:
def plot_latent(z0, z1):
    fig1 = plt.subplots(1,2,figsize=(12,4), constrained_layout=False)
    plt.suptitle ('Colorbar wrt $theta$', fontsize=16)
    plt.subplot(121)
    plt.scatter(z0, z1, c = 90*param_test_norm[:,0], s=4, cmap='jet')
    plt.colorbar()
    plt.xlabel('$z_0$', fontsize=14)
    plt.ylabel('$z_1$', fontsize=14)
    plt.title('In Cartesian coordinates', fontsize=14)
    plt.subplot(122)
    plt.scatter(np.sqrt(z0**2+z1**2), np.arctan2(z1, z0), c = param_test_norm[:,0], s=4, cmap='jet')
    plt.colorbar()
    plt.xlabel('$r$', fontsize=14)
    plt.ylabel(r'$\theta$', fontsize=14)
    plt.title('In polar coordinates', fontsize=14)

    fig2 = plt.subplots(1,2,figsize=(12,4), constrained_layout=False)
    plt.suptitle ('Colorbar wrt $Intensity$', fontsize=16)
    plt.subplot(121)
    plt.scatter(z0, z1, c = (3*param_test_norm[:,1]+1), s=2, cmap='viridis')
    plt.colorbar()
    plt.xlabel('$z_0$', fontsize=14)
    plt.ylabel('$z_1$', fontsize=14)
    plt.title('In Cartesian coordinates$', fontsize=14)
    plt.subplot(122)
    plt.scatter(np.sqrt(z0**2+z1**2), np.arctan2(z1, z0), c = param_test_norm[:,1], s=2, cmap='viridis')
    plt.colorbar()
    plt.xlabel('$r$', fontsize=14)
    plt.ylabel(r'$\theta$', fontsize=14)
    plt.title('In polar coordinates', fontsize=14)
    plt.show()  
    plt.close()

In [ ]:
plot_latent(z_mean[:, 0], z_mean[:,1])

##Function to draw the test and reconstructed examples

In [ ]:
def plot_examples(i1, i2):    
    fig = plt.subplots(2,1,figsize=(12,8),constrained_layout=False)
    plt.suptitle('Examples: Test points and prediction', fontsize=16)
    plt.subplot(211)
    plt.title('theta = ' + str(round(param_test[i1,0]*180/PI,2)) + ", I = " + str(round((param_test[i1,1]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.scatter(t_n, y_test[i1], color="blue", s = 0.5)
    plt.plot(t_n, prediction[i1], color="red", linewidth = 1)
    plt.ylabel('$d(t)$, a.u.', fontsize=16)
    plt.xlim(0, 20)
    plt.xticks(np.arange(0, Tmax, 2.0))
    plt.grid()
    plt.subplot(212)
    plt.title('theta = ' + str(round(param_test[i2,0]*180/PI,2)) + ", I = " + str(round((param_test[i2,1]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.scatter(t_n, y_test[i2], color="blue", s = 0.5)
    plt.plot(t_n, prediction[i2], color="red", linewidth = 1)  
    plt.ylabel('$d(t)$, a.u.', fontsize=16)
    plt.xlabel('$t$, fs', fontsize=16)
    plt.xlim(0, 20)
    plt.xticks(np.arange(0, Tmax, 2.0))
    plt.grid()  
    plt.show() 
    plt.close() 

##Comparison of arbitrary $y_{test}$ (blue) and $y_{reconstructed}$ (red)

In [ ]:
i_show1 = np.random.randint(0, n_test-1)
i_show2 = np.random.randint(0, n_test-1)

plot_examples(i_show1, i_show2)